# Start a chat with LLM

The `Branch` object is the foundational building block of lionagi, serving as your primary interface for AI model interactions.

Think of a Branch as a conversation thread that:
- **Manages messages**: Automatically handles system prompts, user inputs, and AI responses in a structured conversation flow
- **Enables multi-turn conversations**: Maintains context across multiple exchanges, allowing for natural back-and-forth dialogue
- **Handles advanced features**: Supports structured outputs, tool usage, and sophisticated context management out of the box

**Architecture Overview:**
- A `Session` can contain multiple branches, but every session starts with a default `'main'` branch
- Each branch maintains its own conversation history in `branch.messages`

This design allows you to organize complex AI workflows into logical conversation threads, each with their own context and purpose.

**Configuration and Model Selection:**

lionagi comes with sensible defaults for popular AI providers. You can explore these configurations in `lionagi.integrations.config` to understand available models and their settings.

The framework supports multiple AI providers through a unified interface, making it easy to switch between different models without changing your core application logic.

### 1. Using `chat()` - Stateless Interactions

The `chat()` method provides **stateless** communication with AI models. Each call is independent, making it perfect for one-off queries or when you want to control conversation history manually.

**When to use `chat()`:**
- Single-shot questions that don't require conversation context
- When you want to manually manage conversation state
- For parallel processing of independent queries

In [1]:
from lionagi import iModel, Branch
from IPython.display import Markdown

In [2]:
SYSTEM = "As a comedian, you are sarcastically funny"
prompt1 = "short joke: a blue whale and a big shark meet at the bar and start dancing"

In [3]:
# create a branch with gpt-4.1-nano model, need an OpenAI API key
comedian1 = Branch(
    chat_model=iModel(model="openai/gpt-4.1-nano"), system=SYSTEM
)
joke1 = await comedian1.chat(prompt1)

Markdown(f"**comedian1**: {joke1}")

**comedian1**: So a blue whale and a big shark walk into a bar, see each other, and suddenly break out into dance. Guess they're just trying to out-swim each other in the dance floor—because nothing says "peak aquatic rivalry" like a synchronized swim… in flip-flops.

**Model Provider Flexibility:**

lionagi's unified interface allows you to seamlessly switch between different AI providers. Whether you're using OpenAI's latest models, local Ollama instances, or other providers, the API remains consistent.

This provider-agnostic approach gives you the flexibility to:
- Choose the best model for your specific use case
- Switch providers based on cost, performance, or availability
- Test the same logic across different AI models

### 2. Using `communicate()` - Stateful Conversations

The `communicate()` method provides **stateful** interactions where each message builds upon the previous conversation history. This is ideal for multi-turn dialogues and complex reasoning tasks.

**Key Difference from `chat()`:**
- `chat()` is **stateless**: Each call is independent, no memory between calls
- `communicate()` is **stateful**: Automatically maintains conversation history, enabling context-aware responses

**When to use `communicate()`:**
- Multi-turn conversations where context matters
- Building up complex reasoning over multiple exchanges
- When you want automatic conversation memory management

In [4]:
# Need to install "lionagi[ollama]" to use Ollama models

comedian2 = Branch(
    chat_model=iModel(model="ollama/deepseek-r1:1.5b"), system=SYSTEM
)
joke2 = await comedian2.communicate(prompt1)

Markdown(f"**comedian2**: {joke2}")

**comedian2**: <think>
Okay, so I need to create a joke for this scenario. The premise is that a blue whale and a big shark meet at the bar and start dancing as if they're both just having fun. However, it's important these two represent a pair of opposites.

First, I should think about the qualities each animal brings. Blue whales are big and have that iconic black fur; they're often associated with luxury and status. Big sharks are also large but might bring the feel of danger or raw anger to the party, especially during the heat.

The issue here is how to combine these two so they don't end up arguing over their similarities. I need a neutral way for them to dance together instead.

I should come up with a situation where both animals find solace in each other's company. Maybe something that highlights their mutual love or support without making them argue.

What emotions could work here? Fear might come to mind when big sharks are around, especially in the heat of dinner or socializing at a bar. But that could create some tension at the end.

Another angle is their emotional journey—exploiting and then surrendering by dancing together. It shows mutual support but also highlights their contrasting traits over time. That seems relatable.

Let me structure this as an exchange where each animal acknowledges the other's role in providing comfort or stability.
</think>

Oh, I've had a lot easier since you came to the bar. Maybe next time I'll keep it simple and just come with both my teeth and my legs.

In [5]:
# One message, only the system

comedian1.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,1.753196e+09,system,"{'system_message': 'As a comedian, you are sar...",51f0b9b6-c278-4c8a-996a-3da8c3ad3bdd,system,b7ced6ad-8b83-4ddb-84a4-21b2cc6f0b35,{'lion_class': 'lionagi.protocols.messages.sys...


In [6]:
# Three messages, system + user + assistant

comedian2.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,1.753196e+09,system,"{'system_message': 'As a comedian, you are sar...",670fbe44-1efa-49fe-bbc8-ff08a356c634,system,6eeb5846-5bb0-446f-bec3-f9b8ab10ef65,{'lion_class': 'lionagi.protocols.messages.sys...
1,1.753196e+09,user,"{'context': [], 'instruction': 'short joke: a ...",94f80a80-fcac-41d8-912b-8c06c3874df6,user,6eeb5846-5bb0-446f-bec3-f9b8ab10ef65,{'lion_class': 'lionagi.protocols.messages.ins...
2,1.753196e+09,assistant,"{'assistant_response': '<think> Okay, so I nee...",44a82dd2-02e5-47a0-96fc-3636af5ada0a,6eeb5846-5bb0-446f-bec3-f9b8ab10ef65,user,"{'model_response': {'id': 'chatcmpl-532', 'obj..."


In [7]:
# Review the entire model response from LLMs

comedian2.messages[-1].metadata

{'model_response': {'id': 'chatcmpl-532',
  'object': 'chat.completion',
  'created': 1753196352,
  'model': 'deepseek-r1:1.5b',
  'system_fingerprint': 'fp_ollama',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': "<think>\nOkay, so I need to create a joke for this scenario. The premise is that a blue whale and a big shark meet at the bar and start dancing as if they're both just having fun. However, it's important these two represent a pair of opposites.\n\nFirst, I should think about the qualities each animal brings. Blue whales are big and have that iconic black fur; they're often associated with luxury and status. Big sharks are also large but might bring the feel of danger or raw anger to the party, especially during the heat.\n\nThe issue here is how to combine these two so they don't end up arguing over their similarities. I need a neutral way for them to dance together instead.\n\nI should come up with a situation where both animals find solace i